In [0]:
import os, warnings

warnings.filterwarnings(action='ignore')

In [46]:
import tensorflow as tf
import numpy as np
from keras.utils import np_utils

categories = ["top", "pants"]
nb_classes = len(categories)

# 데이터 불러오기
def load_dataset():
  x_train, x_test, y_train, y_test = np.load("clothes_1.npy", allow_pickle=True)
  x_train = x_train.astype("float") / 256
  x_test = x_test.astype("float") / 256
  y_train = np_utils.to_categorical(y_train, nb_classes)
  y_test = np_utils.to_categorical(y_test, nb_classes)
  return x_train, x_test, y_train, y_test

X_train, X_test, y_train, y_test = load_dataset()

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1422, 64, 64, 3)
(1422, 2)
(475, 64, 64, 3)
(475, 2)


In [0]:
# import keras

# y_train = keras.utils.to_categorical(y_train, nb_classes)
# y_test = keras.utils.to_categorical(y_test, nb_classes)

# print(y_train.shape)
# print(y_test.shape)

In [0]:
X = tf.placeholder(tf.float32, [None, 64, 64, 3])
Y = tf.placeholder(tf.float32, [None, 2])
keep_prob = tf.placeholder(tf.float32)

In [49]:
print(X)

Tensor("Placeholder_6:0", shape=(?, 64, 64, 3), dtype=float32)


In [50]:
W1 = tf.Variable(tf.random_normal([3, 3, 3, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)

print(W1)
print(L1)

L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
print(L1)

<tf.Variable 'Variable_9:0' shape=(3, 3, 3, 32) dtype=float32_ref>
Tensor("Relu_6:0", shape=(?, 64, 64, 32), dtype=float32)
Tensor("MaxPool_4:0", shape=(?, 32, 32, 32), dtype=float32)


In [51]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)

print(W2)
print(L2)

L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
print(L2)

<tf.Variable 'Variable_10:0' shape=(3, 3, 32, 64) dtype=float32_ref>
Tensor("Relu_7:0", shape=(?, 32, 32, 64), dtype=float32)
Tensor("MaxPool_5:0", shape=(?, 16, 16, 64), dtype=float32)


In [52]:
W3 = tf.Variable(tf.random_normal([16 * 16 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 16 * 16 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

print(W3)
print(L3)

<tf.Variable 'Variable_11:0' shape=(16384, 256) dtype=float32_ref>
Tensor("dropout_2/mul_1:0", shape=(?, 256), dtype=float32)


In [53]:
W4 = tf.Variable(tf.random_normal([256, 2], stddev=0.01))

print(W4)

model = tf.matmul(L3, W4)

print(model)

<tf.Variable 'Variable_12:0' shape=(256, 2) dtype=float32_ref>
Tensor("MatMul_5:0", shape=(?, 2), dtype=float32)


In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,
labels=Y))

optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [0]:
init = tf.global_variables_initializer() # 초기화
sess = tf.Session() # 세션 연결
sess.run(init) # 세션 초기화

In [56]:
batch_size = 100

total_batch = int(X_train.shape[0]/ batch_size)
print(total_batch)

epochs = 20

14


In [0]:
# 다음 배치를 읽어오기 위한 next_batch 유틸리티 함수를 정의합니다.
def next_batch(start, num, data, labels):
  data_X = data[start:start+num]
  data_y = labels[start:start+num]
  return np.asarray(data_X), np.asarray(data_y)

In [58]:
print(X_train.shape, y_train.shape)

batch_X, batch_y = next_batch(0, 10, X_train, y_train)
print(batch_X.shape, batch_y.shape)

(1422, 64, 64, 3) (1422, 2)
(10, 64, 64, 3) (10, 2)


In [62]:
total_cost = 0

print("전체 입력 데이터 : {}".format(X_train.shape))
print("전체 출력 데이터 : {}".format(y_train.shape))

for epoch in range(epochs):
  total_cost = 0
  
  for i in range(total_batch):
    batch_Xs, batch_ys = next_batch(batch_size*i, batch_size, X_train, y_train)
    
    _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_Xs, Y: batch_ys, keep_prob: 0.2})
    total_cost += cost_val
    
    if (i==0 or i==total_batch-1):
      print('data_step = {}, Avg. cost = {:.3f}'.format(i, cost_val))

  print('epoch: {} total.cost = {:.3f}'.format(epoch, total_cost))

전체 입력 데이터 : (1422, 64, 64, 3)
전체 출력 데이터 : (1422, 2)
data_step = 0, Avg. cost = 0.693
data_step = 13, Avg. cost = 0.607
epoch: 0 total.cost = 9.243
data_step = 0, Avg. cost = 0.634
data_step = 13, Avg. cost = 0.580
epoch: 1 total.cost = 8.866
data_step = 0, Avg. cost = 0.600
data_step = 13, Avg. cost = 0.340
epoch: 2 total.cost = 7.057
data_step = 0, Avg. cost = 0.423
data_step = 13, Avg. cost = 0.278
epoch: 3 total.cost = 5.418
data_step = 0, Avg. cost = 0.431
data_step = 13, Avg. cost = 0.202
epoch: 4 total.cost = 4.568
data_step = 0, Avg. cost = 0.336
data_step = 13, Avg. cost = 0.223
epoch: 5 total.cost = 4.100
data_step = 0, Avg. cost = 0.266
data_step = 13, Avg. cost = 0.160
epoch: 6 total.cost = 3.428
data_step = 0, Avg. cost = 0.265
data_step = 13, Avg. cost = 0.201
epoch: 7 total.cost = 3.192
data_step = 0, Avg. cost = 0.220
data_step = 13, Avg. cost = 0.159
epoch: 8 total.cost = 3.030
data_step = 0, Avg. cost = 0.217
data_step = 13, Avg. cost = 0.170
epoch: 9 total.cost = 2.95

In [63]:
print(X_test.shape, y_test.shape)

(475, 64, 64, 3) (475, 2)


In [64]:
model

<tf.Tensor 'MatMul_5:0' shape=(?, 2) dtype=float32>

In [70]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print('정확도:', sess.run(accuracy, feed_dict={X: X_train, Y: y_train, keep_prob: 1.}))

정확도: 0.9556962


In [71]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print('정확도:', sess.run(accuracy, feed_dict={X: X_test, Y: y_test, keep_prob: 1.}))

정확도: 0.9410526
